# Neighhborhood Battle
### -Tanay Tewar

### Week 1- Initial notebook

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
print(' Hello Capstone Project Course!')

 Hello Capstone Project Course!


### Week 3 Part 1- Scraping and cleaning data of neighborhoods in toronto, canada

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto = pd.read_html(url, header=0)
toronto_df=toronto[0]
toronto_df=toronto_df.groupby(toronto_df['Postal Code']).sum()
toronto_df

,Borough,Neighbourhood
Postal Code,,
M1A,Not assigned,Not assigned
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
...,...,...
M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
M9W,Etobicoke,"Northwest, West Humber - Clairville"
M9X,Not assigned,Not assigned


In [4]:
toronto_df=toronto_df[toronto_df['Borough']!='Not assigned']
toronto_df

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."


- Dataframe has been grouped by postal code
- Only those niegnbourhoods are taken which ahve a borough assigned
- Neighbourhoods with same postal code are written together, seperated by a comma


In [5]:
toronto_df.shape

(103, 2)

### Week 3 Part 2 -  Adding latitude and longitude

In [6]:
!pip install geocoder

In [7]:
toronto_df.reset_index(level=0, inplace=True) 

In [8]:
toronto_df

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [9]:
toronto_df.columns

Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')

In [10]:
toronto_df.to_csv("Neighborhood_data.csv")

In [11]:
toronto_coord=pd.read_csv('Geospatial_Coordinates.csv')
toronto_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
toronto_df=toronto_df.merge(toronto_coord, how='inner', on='Postal Code')
toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
toronto_df.shape

(103, 5)

### Week 3 Part 3 - Data analysis and clustering

In [14]:
toronto_df.dtypes

Postal Code       object
Borough           object
Neighbourhood     object
Latitude         float64
Longitude        float64
dtype: object

The clustering of neighborhoods is based on postal code.

  In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information.

Postal codes are of type string.

The neighborhoods are grouped based on the most common venues and accordingly clusters are formed using latitude, longitude, neighborhood and cluster label information.

For Example you may come across few neighborhoods having most common venue as Park.

Then such neighborhoods are grouped under 1 cluster.


In [24]:
import folium 
from geopy.geocoders import Nominatim

In [26]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
tlatitude = location.latitude
tlongitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(tlatitude, tlongitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [27]:
latitude=toronto_df['Latitude'].tolist()
longitude=toronto_df['Longitude'].tolist()

In [28]:
map_toronto = folium.Map(location=[tlatitude, tlongitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        #parse_html=False
        ).add_to(map_toronto)  

map_toronto

##### Using Foursquare API

In [29]:
CLIENT_ID = 'YHAWPQFWFT4E0IJ00AR0OCC3Z4ZJ03SDGLIYI3ABQIOOIMJO' # your Foursquare ID
CLIENT_SECRET = 'TB22VCQESL2QACHVPMYKISKCZ1UYQZO20OPR5M4WOOAGHT4S' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
#LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YHAWPQFWFT4E0IJ00AR0OCC3Z4ZJ03SDGLIYI3ABQIOOIMJO
CLIENT_SECRET:TB22VCQESL2QACHVPMYKISKCZ1UYQZO20OPR5M4WOOAGHT4S
